In [4]:
import os
from urllib.request import urlopen, Request
from urllib.error import HTTPError
from zipfile import ZipFile, BadZipFile
from datetime import datetime

In [5]:
class Data:
    def __init__(self, data_directory="data", need_update="False"):
        self.data_directory = data_directory
        self.need_update = need_update
        self.csv_filepath = ""

    def Download(self):
        try:
            if not os.path.isdir(self.data_directory):
                print(f"Directory '{self.data_directory}' does not exist. Creating.")
                os.mkdir(self.data_directory)
            
            
            data_url = "https://archive.ics.uci.edu/static/public/235/individual+household+electric+power+consumption.zip"
            request = Request(data_url, method="GET")
            responce = urlopen(request)

            if responce.status != 200:
                raise RuntimeError("Server respond with not 200")
            
            raw_data = responce.read()
            zip_data = ZipFile(raw_data)

            if len(zip_data.filelist) != 1:
                raise RuntimeError("One file in zip archive exptected")

            zip_data_file = zip_data.open(zip_data.filelist[0])

            curr_datetime = datetime.now().strftime("%Y%m%d%H%M%S")
            data_filename = "household_power_consumption"
            csv_filename = f"{data_filename}_{curr_datetime}.csv"
            csv_filepath = os.path.join(self.data_directory, csv_filename)

            csv_file = open(csv_filepath, 'wb')
            csv_file.write(zip_data_file.read())

            csv_file.close()
            zip_data_file.close()
            zip_data.close()

        except HTTPError as e:
            print(f"Error code: {e.code}")
            print(e.read())
        except BadZipFile as e:
            print(e)
        except RuntimeError as e:
            print(e)


In [6]:
data = Data()
data.Download()

AttributeError: 'bytes' object has no attribute 'seek'